In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
import statsmodels.api as sm
from scipy.special import gamma, loggamma, factorial
import scipy.stats
from scipy.interpolate import interp1d
from matplotlib.ticker import AutoMinorLocator 
from matplotlib import rc, font_manager
from matplotlib.lines import Line2D
from matplotlib import colors as mcolors
from mycolorpy import colorlist as mcp
from matplotlib import legend_handler
from mpl_toolkits.axes_grid1 import make_axes_locatable, axes_size
import collections
import matplotlib
import shapely.geometry
from shapely.geometry import Point
import shapely.ops 
from geopy.geocoders import Nominatim
import geopandas as gpd
import mapclassify
import jenkspy
import contextily as cx
import os
import rioxarray as rx
import pyreadr
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
# import plots
import seaborn as sns
from datetime import datetime

In [2]:

country = 'Mexico'

if country=='Argentina':
    country_short = 'ARG'
    country_code = 'AR'
elif country == 'Chile':
    country_short = 'CHL'
    country_code = 'CL'
elif country == 'Colombia':
    country_short = 'COL'
    country_code = 'CO'
elif country == 'Mexico':
    country_short = 'MEX'
    country_code = 'MX'


directory = '/Volumes/RECAST/data/outputs/latin-mobility-covid/' + country_short
wd = '/Users/carmen/Library/CloudStorage/OneDrive-TheUniversityofLiverpool/Research/RECAST/latin-mobility-covid-local-files/data/outputs/' + country_short 



files = sorted(os.listdir(wd + '/mov/'))
files = [file for file in files if not file.startswith('.')]

In [3]:
len(files)

762

In [5]:

# Days of week coded according to datetime library (0 is Monday, 6 is Sunday)
### ONLY UNCOMMENT THE LINES BELOW TO FILL IN BASELINE DATAFRAME FROM SCRATCH!!
### CONSIDER: DO YOU WANT TO WORK WITH FILES LOCALLY (WD) OR REMOTELY (DIRECTORY)
df_bl = pd.DataFrame({'O': [], 'D': [], '0': [], '1': [], '2': [], '3': [], '4': [], '5': [], '6': []})
# # df_bl.to_csv(directory + '/baseline/baseline_mov.csv', index=False)
# # filename = directory + '/baseline/counter_mov.txt'
df_bl.to_csv(wd + '/baseline/baseline_mov.csv', index=False)
filename = wd + '/baseline/counter_mov.txt'
with open(filename, 'w') as file:
    file.write(str(0))
    

In [6]:
### CONSIDER: DO YOU WANT TO WORK WITH FILES LOCALLY (WD) OR REMOTELY (DIRECTORY)
### movement files will always be in "directory"

filename = wd + '/baseline/counter_mov.txt'
with open(filename, 'r') as file:
    start_i = file.read()
print(start_i)

df_bl = pd.read_csv(wd + '/baseline/baseline_mov.csv')

for i in range(int(start_i), len(files)):
    
    if i%20 == 0:
        print(i/len(files)*100)
    
    df_bl = pd.read_csv(wd + '/baseline/baseline_mov.csv')

    file = files[i]
    df_movs = pd.read_csv(wd + '/mov/' + file, index_col=0)

    df_movs_bl = df_movs#.dropna(subset=['n_baseline']).reset_index(drop=True)
    df_movs_bl = df_movs_bl.dropna(subset=['start_FID']).reset_index(drop=True)
    df_movs_bl = df_movs_bl.dropna(subset=['end_FID']).reset_index(drop=True)
#     df_movs_bl = df_movs_bl[df_movs_bl['length_km'] < 70].reset_index(drop=True)
    wday = datetime.strptime(file[-19:-9], "%Y-%m-%d").weekday()

    for j in range(len(df_movs_bl)):

        O, D = int(df_movs_bl.loc[j, 'start_FID']), int(df_movs_bl.loc[j, 'end_FID'])

        df_bl_t = df_bl[(df_bl['O']==O) & (df_bl['D']==D)]
        value = df_movs_bl.loc[j, 'n_baseline']

        if len(df_bl_t) == 0:
            row = [int(O), int(D)] + [-999 if k != wday else value for k in range(7)]
            df_bl.loc[len(df_bl)] = row

        else:
            if len(df_bl_t.index) == 1:
                if (df_bl.loc[df_bl_t.index[0], str(wday)] > 0) == False:
                    df_bl.loc[df_bl_t.index[0], str(wday)] = value
            else:
                print('more than one rows in df_bl_t')
            
    filename = wd + '/baseline/counter_mov.txt'
    with open(filename, 'w') as file:
        file.write(str(i))
        
    df_bl.to_csv(wd + '/baseline/baseline_mov.csv', index=False)


0
0.0
2.6246719160104988
5.2493438320209975
7.874015748031496
10.498687664041995
13.123359580052494
15.748031496062993
18.37270341207349
20.99737532808399
23.62204724409449
26.246719160104988
28.871391076115486
31.496062992125985
34.120734908136484
36.74540682414698
39.37007874015748
41.99475065616798
44.61942257217848
47.24409448818898
49.86876640419948
52.493438320209975
55.118110236220474
57.74278215223097
60.367454068241464
62.99212598425197
65.61679790026247
68.24146981627297
70.86614173228347
73.49081364829397
76.11548556430446
78.74015748031496
81.36482939632546
83.98950131233596
86.61417322834646
89.23884514435696
91.86351706036746
94.48818897637796
97.11286089238845
99.73753280839895


In [7]:
i

761

In [8]:
len(files)

762

In [9]:
df_bl

,O,D,0,1,2,3,4,5,6
0,1134.0,959.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,786.0,959.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,871.0,959.0,11.2,NaN,10.4,NaN,13.2,10.25,NaN
3,5594.0,6233.0,NaN,NaN,NaN,NaN,NaN,13.25,NaN
4,6400.0,6400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
142169,1346.0,1343.0,-999.0,-999.0,NaN,-999.0,-999.0,-999.00,-999.0
142170,1083.0,1806.0,-999.0,-999.0,NaN,-999.0,-999.0,-999.00,-999.0
142171,1706.0,1278.0,-999.0,-999.0,NaN,-999.0,-999.0,-999.00,-999.0
142172,1431.0,1288.0,-999.0,-999.0,NaN,-999.0,-999.0,-999.00,-999.0


In [9]:
str(int(df_movs_bl.loc[0, 'start_FID']))

'4516'